In [ ]:
from google.colab import files
uploaded = files.upload()

Saving train.csv to train.csv
Saving test.csv to test.csv


# 문제 정의
미국의 인구조사 데이터(1994)를 바탕으로 만들어진 데이터다. 이 데이터에서 각 사람의 소득을 예측하면 된다. 나이, 결혼 여부, 직종 등의 컬럼이 있다.
- 레이블(타깃)은 연소득이 50,000달러 이상과 미만으로 구분됨(컬럼명: income)
- 소득 예측값(50,000달러 이상일 확률)을 csv 파일로 생성
- 평가 기준은 ROC-AUC로 평가
- 제출 파일은 예측값만 result.csv파일로 생성해 제출(컬럼명: pred. 1개)

# Chapter 2 머신러닝 실습(분류)

In [ ]:
# 데이터 불러오기
import pandas as pd
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [ ]:
#EDA - 데이터 샘플 확인
train.sample(3) #head나 tail도 가능

,id,age,workclass,fnlwgt,education,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country,income
17235,11579,59.0,Self-emp-inc,255822,HS-grad,9,Divorced,Exec-managerial,Not-in-family,White,Male,0,0,60.0,United-States,<=50K
12701,29931,50.0,Private,123174,Doctorate,16,Married-civ-spouse,Prof-specialty,Husband,Black,Male,0,0,37.0,NaN,>50K
745,9211,31.0,Private,273818,Some-college,10,Married-civ-spouse,Other-service,Husband,White,Male,0,0,55.0,Mexico,<=50K


In [ ]:
#데이터 크기
train.shape # 괄호가 없다
# print(train.describe(include='O'))
# test.describe(include='O')

(29304, 16)

In [ ]:
#자료형 확인
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29304 entries, 0 to 29303
Data columns (total 16 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id              29304 non-null  int64  
 1   age             29292 non-null  float64
 2   workclass       27642 non-null  object 
 3   fnlwgt          29304 non-null  int64  
 4   education       29304 non-null  object 
 5   education.num   29304 non-null  int64  
 6   marital.status  29304 non-null  object 
 7   occupation      27636 non-null  object 
 8   relationship    29304 non-null  object 
 9   race            29304 non-null  object 
 10  sex             29304 non-null  object 
 11  capital.gain    29304 non-null  int64  
 12  capital.loss    29304 non-null  int64  
 13  hours.per.week  29291 non-null  float64
 14  native.country  28767 non-null  object 
 15  income          29304 non-null  object 
dtypes: float64(2), int64(5), object(9)
memory usage: 3.6+ MB


In [ ]:
# 기초통계량 확인
train.describe()

,id,age,fnlwgt,education.num,capital.gain,capital.loss,hours.per.week
count,29304.000000,29292.000000,2.930400e+04,29304.000000,29304.000000,29304.000000,29291.000000
mean,16264.027880,38.553223,1.897488e+05,10.080842,1093.858722,86.744506,40.434229
std,9384.518323,13.628811,1.055250e+05,2.570824,7477.435640,401.518928,12.324036
min,0.000000,-38.000000,1.228500e+04,1.000000,0.000000,0.000000,1.000000
25%,8145.750000,28.000000,1.177890e+05,9.000000,0.000000,0.000000,40.000000
50%,16253.500000,37.000000,1.783765e+05,10.000000,0.000000,0.000000,40.000000
75%,24374.250000,48.000000,2.370682e+05,12.000000,0.000000,0.000000,45.000000
max,32560.000000,90.000000,1.484705e+06,16.000000,99999.000000,4356.000000,99.000000


In [ ]:
# 기초통계량 확인 - 범주형
train.describe(include='object')
# train.describe(include='O') # 위의 함수와 같은 의미를 나타낸다.

,workclass,education,marital.status,occupation,relationship,race,sex,native.country,income
count,27642,29304,29304,27636,29304,29304,29304,28767,29304
unique,8,16,7,14,6,5,2,41,2
top,Private,HS-grad,Married-civ-spouse,Prof-specialty,Husband,White,Male,United-States,<=50K
freq,20451,9449,13466,3683,11845,25022,19578,26240,22263


# 결측치 전처리

In [ ]:
# 결측치 확인 - train
train.isnull().sum() # 결측치 개수 확인

,0
id,0
age,12
workclass,1662
fnlwgt,0
education,0
education.num,0
marital.status,0
occupation,1668
relationship,0
race,0


In [ ]:
# 결측치 확인 - test
test.isnull().sum()

,0
id,0
age,6
workclass,174
fnlwgt,0
education,0
education.num,0
marital.status,0
occupation,175
relationship,0
race,0


In [ ]:
# traget 빈도수 확인
train['income'].value_counts() # 이진 분류여부와 불균형 데이터 여부 확인(양호한 정도임)

,count
income,
<=50K,22263
>50K,7041


In [ ]:
# 결측치 처리
# 일반적으로 train값에 바로 저장하지만, 여기서는 df로 따로 저장해 제거된 갯수를 확인한다.

print("전처리전:", train.shape)
df = train.dropna()
print('전처리후:', df.shape)

전처리전: (29304, 16)
전처리후: (27096, 16)


In [ ]:
# 특정 컬럼에서 결측치가 있는 경우, 그 데이터의 행을 처리
df = train.dropna(subset = ['native.country', 'workclass'])
print(df.isnull().sum())
df.isnull().sum()

id                 0
age               12
workclass          0
fnlwgt             0
education          0
education.num      0
marital.status     0
occupation         6
relationship       0
race               0
sex                0
capital.gain       0
capital.loss       0
hours.per.week    13
native.country     0
income             0
dtype: int64


,0
id,0
age,12
workclass,0
fnlwgt,0
education,0
education.num,0
marital.status,0
occupation,6
relationship,0
race,0


In [ ]:
# 특정 컬럼에서 결측치가 있는 경우, 그 데이터의 column을 처리
df = train.dropna(axis = 1) #axis=1로 열로 지정
df.shape

(29304, 11)

In [ ]:
# 특정 컬럼만 삭제
df = train.drop(['native.country', 'workclass'], axis=1)
df.shape

(29304, 14)

In [ ]:
# 결측치 채우기(범주형)
# 범주형은 주로 mode로 채운다
train.isnull().sum()

# 최빈값 탐색
m1 = train['workclass'].mode()[0]
m2 = train['native.country'].mode()[0]

#결측값 채우기
train['workclass'] = train['workclass'].fillna(m1)
train['native.country'] = train['native.country'].fillna(m2)

#결측값 채운 여부 확인
train.isnull().sum()

,0
id,0
age,12
workclass,0
fnlwgt,0
education,0
education.num,0
marital.status,0
occupation,1668
relationship,0
race,0


In [ ]:
# 결측치를 unique한 별도의 결측 데이터로 채우기
train['occupation'] = train['occupation'].fillna('X')
train.isnull().sum()

,0
id,0
age,12
workclass,0
fnlwgt,0
education,0
education.num,0
marital.status,0
occupation,0
relationship,0
race,0


In [ ]:
# test데이터에도 동일하게 적용(삭제하는 것이 아니라면)

test['workclass'] = test['workclass'].fillna(test['workclass'].mode()[0])
test['native.country'] = test['native.country'].fillna(test['native.country'].mode()[0])
test['occupation'] = test['occupation'].fillna('X')

test.isnull().sum()

,0
id,0
age,6
workclass,0
fnlwgt,0
education,0
education.num,0
marital.status,0
occupation,0
relationship,0
race,0


In [ ]:
#결측치 채우기(수치형)

# 평균값으로 채우기
value = int(train['age'].mean())
print("평균값:",value)
train['age'] = train['age'].fillna(value)
test['age'] = test['age'].fillna(value)

평균값: 38


> **중요: test데이터의 결측값은 과대학습을 방지하기 위해서 train데이터의 통계량을 사용해야 한다.**

In [ ]:
# 중강값으로 채우기
value = int(train['hours.per.week'].median())
print('중앙값:', value)
train['hours.per.week'] = train['hours.per.week'].fillna(value)
test['hours.per.week'] = test['hours.per.week'].fillna(value)

중앙값: 40


In [ ]:
#결측치 없는거 확인
train.isnull().sum()

,0
id,0
age,0
workclass,0
fnlwgt,0
education,0
education.num,0
marital.status,0
occupation,0
relationship,0
race,0


# 이상치 처리
  이상치 처리는 안해도 되나, 일반적인 상식선에서 말 안되는 것을 처리하자. (ex. 음수나이)

In [ ]:
#기술통계량 확인
train.describe()

# age 음수 (이상치) 확인

,id,age,fnlwgt,education.num,capital.gain,capital.loss,hours.per.week
count,29304.000000,29304.000000,2.930400e+04,29304.000000,29304.000000,29304.000000,29304.000000
mean,16264.027880,38.552996,1.897488e+05,10.080842,1093.858722,86.744506,40.434036
std,9384.518323,13.626025,1.055250e+05,2.570824,7477.435640,401.518928,12.321306
min,0.000000,-38.000000,1.228500e+04,1.000000,0.000000,0.000000,1.000000
25%,8145.750000,28.000000,1.177890e+05,9.000000,0.000000,0.000000,40.000000
50%,16253.500000,37.000000,1.783765e+05,10.000000,0.000000,0.000000,40.000000
75%,24374.250000,48.000000,2.370682e+05,12.000000,0.000000,0.000000,45.000000
max,32560.000000,90.000000,1.484705e+06,16.000000,99999.000000,4356.000000,99.000000


In [ ]:
# 이상치 확인
train[train['age']<=0]

,id,age,workclass,fnlwgt,education,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country,income
39,29188,-33.0,Private,263561,Some-college,10,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,60.0,United-States,>50K
79,14325,-38.0,Private,22245,HS-grad,9,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,60.0,United-States,>50K
26161,4292,-25.0,Private,200681,Some-college,10,Never-married,X,Own-child,White,Male,0,0,40.0,United-States,<=50K


In [ ]:
# 이상치 제거
train = train[train['age']>0]
print(train.shape)

(29301, 16)


# 인코딩
범주형 데이터를 수치형 데이터로 바꾸는 과정
> traget이 아닌, 독립변수를 변형한다.

> .pop()을 통해 변수를 제거하여 반환가능.

1. 원-핫 인코딩

  pd.get_dummies()로 원-핫 인코딩 자동 수행 가능(object만 자동 실행)

2. 레이블 인코딩
  
  사이킷런 LabelEncoder로 칼럼별로 개별적으로 적용해야함.

  > 반복문사용, 코드 참고

  > https://colab.research.google.com/drive/1IziROYvvXjQsfwNzfvseHXL5HL3ANrN-?authuser=2#scrollTo=hxl32jLoY4tY&line=8&uniqifier=1

In [ ]:
# label을 일단 별도로 옮긴다.
y_train = train.pop('income')
y_train

,income
0,>50K
1,<=50K
2,<=50K
3,>50K
4,<=50K
...,...
29299,<=50K
29300,<=50K
29301,<=50K
29302,<=50K


In [ ]:
# 원-핫 인코딩
train_oh = pd.get_dummies(train)
test_oh = pd.get_dummies(test)
print(train.shape, test.shape, train_oh.shape, test_oh.shape)

(29301, 15) (3257, 15) (29301, 107) (3257, 103)


In [ ]:
# 칼럼수 일치시키는 원-핫 인코딩(필수)
data = pd.concat([train, test], axis=0)
data_oh = pd.get_dummies(data)
# 실제시험에서는 바로 train, test변수에 옮기기
train_oh = data_oh.iloc[:len(train)].copy()
test_oh = data_oh.iloc[len(train):].copy()

print(train_oh.shape, test_oh.shape)

(29301, 107) (3257, 107)


In [ ]:
# 레이블 인코딩
# object만 사용하는 코드를 선택하거나 cols 변수에 리스트 형태로 직접 제적
# # 방법1
# cols = train.select_dtypes(include = 'O').columns
# cols

# 방법 2
# cols = train.columns[train.dtypes == object]
# cols

# 방법 3 (추천)
cols = ['workclass', 'education','marital.status','occupation','relationship','race',
        'sex', 'native.country']
cols

['workclass',
 'education',
 'marital.status',
 'occupation',
 'relationship',
 'race',
 'sex',
 'native.country']

In [ ]:
# 레이블 인코딩(사이킷런 LabelEncoder)
from sklearn.preprocessing import LabelEncoder
for col in cols:
  le = LabelEncoder()
  train[col] = le.fit_transform(train[col]) # fit은 매핑
  test[col] = le.transform(test[col]) # 이미 매핑됐으니 transform만 적용

train.head()

,id,age,workclass,fnlwgt,education,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country
0,3331,34.0,6,177331,15,10,2,9,0,2,1,4386,0,40.0,38
1,19749,58.0,3,290661,11,9,2,2,0,4,1,0,0,40.0,38
2,1157,48.0,3,125933,15,10,6,3,4,2,0,0,1669,38.0,38
3,693,58.0,3,100313,15,10,2,10,0,4,1,0,1902,40.0,38
4,12522,41.0,3,195661,15,10,2,13,0,4,1,0,0,54.0,38


# 스케일링

데이터 범위를 조정하는 작업(수치형 데이터)

사이킷런의 Min-Max Scaling, Standard Scaling, Robust Scaling


| 메서드               | 기능 요약                           | 공통 의미               |
| ----------------- | ------------------------------- | ------------------- |
| `fit()`           | **훈련 데이터 기반 통계/정보 학습**          | 평균, 범위, 클래스 목록 등 학습 |
| `transform()`     | **학습된 정보 기반으로 데이터 변환**          | 스케일 조정, 숫자 인코딩 등 수행 |
| `fit_transform()` | `fit()` + `transform()` 한 번에 수행 | 빠르게 학습하고 바로 변환      |


| 적용 대상                                       | `fit()`이 학습하는 것                        | `transform()`이 하는 것 |
| ------------------------------------------- | -------------------------------------- | ------------------- |
| **스케일링** (`StandardScaler`, `MinMaxScaler`) | 평균/표준편차 또는 최소/최대값 계산                   | 정규화된 값으로 변환         |
| **LabelEncoder**                            | 고유 클래스 목록 (예: \['cat', 'dog', 'bird']) | 각 클래스에 대응하는 정수로 인코딩 |


In [ ]:
# 스케일링을 사용할 수치형 컬럼명 리스트화
cols = ['age', 'fnlwgt', 'education.num', 'capital.gain', 'capital.loss', 'hours.per.week']

['age',
 'fnlwgt',
 'education.num',
 'capital.gain',
 'capital.loss',
 'hours.per.week']

In [ ]:
# 사본 데이터 제작(scaling별 비교)
def get_data():
  train_copy = train.copy()
  test_copy = test.copy()
  return train_copy, test_copy

<class 'pandas.core.frame.DataFrame'>
Index: 29301 entries, 0 to 29303
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id              29301 non-null  int64  
 1   age             29301 non-null  float64
 2   workclass       29301 non-null  int64  
 3   fnlwgt          29301 non-null  int64  
 4   education       29301 non-null  int64  
 5   education.num   29301 non-null  int64  
 6   marital.status  29301 non-null  int64  
 7   occupation      29301 non-null  int64  
 8   relationship    29301 non-null  int64  
 9   race            29301 non-null  int64  
 10  sex             29301 non-null  int64  
 11  capital.gain    29301 non-null  int64  
 12  capital.loss    29301 non-null  int64  
 13  hours.per.week  29301 non-null  float64
 14  native.country  29301 non-null  int64  
dtypes: float64(2), int64(13)
memory usage: 3.6 MB
None


In [ ]:
# Min-Max Scaling
train_copy, test_copy = get_data()

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
display(train_copy[cols].head(2))
train_copy[cols] = scaler.fit_transform(train_copy[cols])
test_copy[cols] = scaler.transform(test_copy[cols])
display(train_copy[cols].head(2))

,age,fnlwgt,education.num,capital.gain,capital.loss,hours.per.week
0,34.0,177331,10,4386,0,40.0
1,58.0,290661,9,0,0,40.0


,age,fnlwgt,education.num,capital.gain,capital.loss,hours.per.week
0,0.232877,0.112092,0.600000,0.04386,0.0,0.397959
1,0.561644,0.189060,0.533333,0.00000,0.0,0.397959


In [ ]:
# Standard Scaling(정규화)

train_copy, test_copy = get_data()

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
display(train_copy[cols].head())

train_copy[cols] = scaler.fit_transform(train_copy[cols])
test_copy[cols] = scaler.transform(test_copy[cols])
display(train_copy[cols].head())

,age,fnlwgt,education.num,capital.gain,capital.loss,hours.per.week
0,34.0,177331,10,4386,0,40.0
1,58.0,290661,9,0,0,40.0
2,48.0,125933,10,0,1669,38.0
3,58.0,100313,10,0,1902,40.0
4,41.0,195661,10,0,0,54.0


,age,fnlwgt,education.num,capital.gain,capital.loss,hours.per.week
0,-0.335121,-0.117705,-0.031462,0.440247,-0.216056,-0.035121
1,1.428590,0.956277,-0.420430,-0.146298,-0.216056,-0.035121
2,0.693710,-0.604783,-0.031462,-0.146298,3.940528,-0.197450
3,1.428590,-0.847573,-0.031462,-0.146298,4.520806,-0.035121
4,0.179295,0.056001,-0.031462,-0.146298,-0.216056,1.101181


In [ ]:
# Robust Scaling
# (X - Median) / IQR
# 이상치 영향을 덜받음

from sklearn.preprocessing import RobustScaler
scaler = RobustScaler()
display(train[cols].head(2))
train[cols] = scaler.fit_transform(train[cols])
test[cols] = scaler.transform(test[cols])
display(train[cols].head(2))

,age,fnlwgt,education.num,capital.gain,capital.loss,hours.per.week
0,34.0,177331,10,4386,0,40.0
1,58.0,290661,9,0,0,40.0


,age,fnlwgt,education.num,capital.gain,capital.loss,hours.per.week
0,-0.15,-0.008711,0.000000,4386.0,0.0,0.0
1,1.05,0.941438,-0.333333,0.0,0.0,0.0


In [ ]:
# map(), replace() 등으로 target을 인코딩할 수 도 있음
# 이진분류면 인코딩하고 다중 분류는 그대로 진행하는편
# Xgboost는 반드시 인코딩 해야함

#  mapping
# target = y_train.map({
#     '<=50K':0,
#     '>50K':1
# })
# target.value_counts()

# replace

target = y_train.replace('<=50K', 0).replace('>50K',1)
target.value_counts()

<ipython-input-116-b5140846ee8a>:14: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  target = y_train.replace('<=50K', 0).replace('>50K',1)


,count
income,
0,22262
1,7039


# 검증 데이터 나누기

사이킷런의 train_test_split()을 활용해 데이터 분리

> 그저 함수명일뿐, train과 validation을 나눠도 무방

**train_test_split(train, target, test_size = 0.2, random_state = 0**

train: X데이터(df)

target: y데이터(series)

test_size = Proportion of Validation data

random_state =0 : 랜덤적인 요소를 고정(시드를 의미)

반환값: X_train, X_val, y_train, y_val 으로 4개가 반환

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(train,
                                                  y_train,
                                                  test_size=0.2,
                                                  random_state=0)

X_train.shape, X_val.shape, y_train.shape, y_val.shape

# 데이터 반환 후 x train val의 col이 같아야 함.
# y가 Series로 반환되어야함(열의 개수가 표시 X)

((23440, 15), (5861, 15), (23440,), (5861,))

# 머신러닝 학습 및 평가

의사결정나무에서 앙상블 기법사용해서 보완

[시험 상 추천]

배깅: Random Forest

1. rf = RandomForestClassifier() #모델선택
2. rf.fit(X_train, y_train) #학습
3. pred = rf.predict_proba(X_val) #예측

> rf.predict: 클래스 반환, rf.predict_proba: 클래스별 반환확률

부스팅: LightGBM

평가지표: 정확도, 정밀도, 재현율, F1-score, ROC_AUC

- predict_proba() : roc_auc
- predict() : 정확도, F1 Score, 정밀도, 재현율

[하이퍼파라미터)

RandomForest: n_estimators, max_depth, min_samples_split 등

LGBM: learning_rate, num_leaves, max_depth, n_estimators

In [ ]:
#랜덤 포레스트
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(random_state= 0)
rf.fit(X_train, y_train)
pred = rf.predict_proba(X_val) # 각 레이블에 속할 확률값 반환

print(rf.classes_) # 컬럼의 순서 확인
pred[:10]

['<=50K' '>50K']


array([[1.  , 0.  ],
       [1.  , 0.  ],
       [0.9 , 0.1 ],
       [0.63, 0.37],
       [1.  , 0.  ],
       [0.99, 0.01],
       [0.98, 0.02],
       [0.94, 0.06],
       [0.12, 0.88],
       [0.88, 0.12]])

In [ ]:
# 평가지표
#roc_auc
from sklearn.metrics import roc_auc_score
roc_auc = roc_auc_score(y_val, pred[:,1]) # >50K의 확률
print('roc_auc:', roc_auc)

roc_auc: 0.9173623004487484


In [ ]:
# 정확도

pred = rf.predict(X_val)
pred[:10]
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_val, pred)
print('acuuracy_score:', accuracy)

acuuracy_score: 0.8694761986009213


In [ ]:
# F1스코어
from sklearn.metrics import f1_score
f1 = f1_score(y_val, pred, pos_label='>50K')
print('f1_score:',f1)

f1_score: 0.6926476496584973


In [ ]:
# LightGBM
import lightgbm as lgb
lgbmc = lgb.LGBMClassifier(random_state=0,verbose=-1) #verbose=-1은 오류메시지 숨기기
lgbmc.fit(X_train,y_train)
pred = lgbmc.predict_proba(X_val)

roc_auc = roc_auc_score(y_val, pred[:,1])
print('roc_auc:', roc_auc)

pred = lgbmc.predict(X_val)
accuracy = accuracy_score(y_val,pred)
print('acuuracy_score:', accuracy)

f1 = f1_score(y_val, pred, pos_label = '>50K') #pos_label: 양성인자 지정
print('f1_score:',f1)

roc_auc: 0.9279535666686397
acuuracy_score: 0.8771540692714553
f1_score: 0.7158642462509865


# 예측 및 결과 파일 생성

test파일로 예측

In [ ]:
pred = lgbmc.predict_proba(test)
pred

array([[0.89924007, 0.10075993],
       [0.97622077, 0.02377923],
       [0.9853122 , 0.0146878 ],
       ...,
       [0.93528667, 0.06471333],
       [0.98996872, 0.01003128],
       [0.96723292, 0.03276708]])

In [ ]:
submit = pd.DataFrame({'pred':pred[:,1]})
submit.to_csv('result.csv',index=False)

In [ ]:
pd.read_csv('result.csv')

,pred
0,0.100760
1,0.023779
2,0.014688
3,0.798048
4,0.026542
...,...
3252,0.009613
3253,0.321992
3254,0.064713
3255,0.010031
